In [18]:
import pprint as pprint
import xml.etree.cElementTree as ET
import re
from collections import defaultdict
import time
import codecs
import json
from pymongo import MongoClient

## Load File and Create Sample File

In [2]:
OSM_FILE = "chicago.osm"

In [4]:
#Create a sample file from the full chicago OSM data
SAMPLE_FILE = "chicago_sample.osm"

k = 150

def get_element(OSM_FILE, tags=('node', 'way', 'relation')):
    context = ET.iterparse(OSM_FILE, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()
            
            
with open (SAMPLE_FILE, 'wb') as output:
    output.write('<?xml version="1.0" encoding="UTF-8"?>\n')
    output.write('<osm>\n ')

    for i, element in enumerate(get_element(OSM_FILE)):
        if i % k == 0:
            output.write(ET.tostring(element, encoding='utf-8'))

    output.write('</osm>')

## Key value check

In [15]:
# Before you process the data and add it into MongoDB, you should
# check the "k" value for each "<tag>" and see if they can be valid keys in MongoDB,
# as well as see if there are any other potential problems.

lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

#determine if a tag matches one of the regular expression conditions above 
# and add a count to the dict for that condition

def key_type(element, keys):
    if element.tag == "tag":
       
        for tag in element.iter('tag'):
            tagk = tag.attrib['k']
            if lower.match(tagk):
                keys['lower'] += 1
            elif problemchars.match(tagk):
                keys['problemchars'] += 1
            elif lower_colon.match(tagk):
                keys['lower_colon'] += 1
            #if none of the searches as true add a count to 'other'
            else:
                keys['other'] += 1

    return keys


def process_map(filename):
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
    for _, element in ET.iterparse(filename):
        keys = key_type(element, keys)

    return keys

process_map(SAMPLE_FILE)

{'lower': 13015, 'lower_colon': 20506, 'other': 10680, 'problemchars': 0}

## Tag Type Count | Map Parser

In [16]:
def get_element(filename):
   
    context = ET.iterparse(filename, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end':
            yield elem
            root.clear()


def count_tags (filename):
    tags_dict = defaultdict(int)
    for i, elem in enumerate(get_element(filename)):
        tags_dict[elem.tag] += 1
    return tags_dict

count_tags(SAMPLE_FILE)

defaultdict(int,
            {'member': 212,
             'nd': 65469,
             'node': 55997,
             'osm': 1,
             'relation': 30,
             'tag': 44201,
             'way': 7917})

## User IDs

In [21]:
"""
Your task is to explore the data a bit more.
The first task is a fun one - find out how many unique users
have contributed to the map in this particular area!

The function process_map should return a set of unique user IDs ("uid")
"""

def get_user(element):
    return

def process_map(filename):
    users = set()
    for _, element in ET.iterparse(filename):
        if element.tag == 'node':
            for node in element.iter('node'):
                userid = node.attrib['uid']
                users.add(userid)

    return users


def test():

    users = process_map(SAMPLE_FILE)
    pprint.pprint(users)
    #assert len(users) == 6


test()

set(['100042',
     '1007528',
     '1009211',
     '101842',
     '103253',
     '10353',
     '1051550',
     '1058308',
     '105839',
     '107257',
     '1078496',
     '10786',
     '1087647',
     '11111',
     '111916',
     '1132286',
     '1132565',
     '113823',
     '1138968',
     '114430',
     '1149057',
     '115918',
     '1164',
     '1178825',
     '1179400',
     '1181429',
     '1190212',
     '1196680',
     '1198074',
     '1198553',
     '119881',
     '1200080',
     '120146',
     '120468',
     '121241',
     '1218455',
     '1219059',
     '122940',
     '1237170',
     '1240849',
     '12434',
     '1249130',
     '125718',
     '1260280',
     '126442',
     '128470',
     '129101',
     '1291871',
     '12919',
     '1295',
     '130472',
     '1306',
     '130794',
     '132494',
     '1333285',
     '1350208',
     '135108',
     '135163',
     '135346',
     '135435',
     '1355910',
     '135851',
     '1376118',
     '1376589',
     '1399520',
     

## Number of Unique Users

In [13]:
def get_user(element):
    return

def process_map(filename):
    users = set()
    for _, element in ET.iterparse(filename):
        if element.tag == 'node':
            for node in element.iter('node'):
                userid = node.attrib['uid']
                users.add(userid)

    return users

len(process_map(OSM_FILE))

2005

## Data Audit

In [25]:
"""
Your task in this exercise has two steps:
- audit the OSMFILE and change the variable 'mapping' to reflect the changes needed to fix 
    the unexpected street types to the appropriate ones in the expected list.
    You have to add mappings only for the actual problems you find in this OSMFILE,
    not a generalized solution, since that may and will depend on the particular area you are auditing.
- write the update_name function, to actually fix the street name.
    The function takes a string with street name as an argument and should return the fixed name
    We have provided a simple test so that you see what exactly is expected
"""

street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)


expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons", "Circle", "South Avenue "]

mapping = {"St": "Street", "St.": "Street", "street": "Street","Dr": "Drive", "Dr.": "Drive","Rd": "Road","Ct":"Court"}


def audit_street_type(street_types, street_name):
    """this function audit street name type"""
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)


def is_street_name(elem):
    """this function if an attribute includes a street"""
    return (elem.attrib['k'] == "addr:street")


def audit(osmfile):
    """this function return all street name types"""
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])

    return street_types


def update_name(name, mapping):
    """this function update street name and make street name to be consistent"""
    name = name.replace(",", "")
    for word in name.split(" "):
        if word in mapping.keys():
            name = name.replace(word, mapping[word])

    return name


def output(osmfile):
    """This function print updated street name"""
    st_types = audit(osmfile)
    #assert len(st_types) == 3
    pprint.pprint(dict(st_types))

    for st_type, ways in st_types.iteritems():
        for name in ways:
            better_name = update_name(name, mapping)
            print name, "=>", better_name
    print "Pass"
    
    
output(SAMPLE_FILE)

{'B': set(['South Avenue B']),
 'Broadway': set(['North Broadway']),
 'Ct': set(['Vail Ct']),
 'Dr': set(['Briarwood Dr', 'John M Boor Dr']),
 'E': set(['South Avenue E']),
 'F': set(['South Avenue F']),
 'G': set(['South Avenue G']),
 'H': set(['South Avenue H']),
 'Highway': set(['Lincoln Highway', 'North Northwest Highway']),
 'J': set(['South Avenue J']),
 'K': set(['South Avenue K']),
 'L': set(['South Avenue L']),
 'Ln': set(['White Feather Ln']),
 'M': set(['South Avenue M']),
 'N': set(['South Avenue N']),
 'O': set(['South Avenue O']),
 'Park': set(['East Groveland Park', 'East Madison Park']),
 'St': set(['Tipperary St']),
 'Terrace': set(['North Geneva Terrace'])}
South Avenue B => South Avenue B
South Avenue E => South Avenue E
South Avenue G => South Avenue G
South Avenue F => South Avenue F
White Feather Ln => White Feather Ln
South Avenue M => South Avenue M
South Avenue K => South Avenue K
South Avenue J => South Avenue J
East Groveland Park => East Groveland Park
East 

## Data Setup

In [46]:
"""
Your task is to wrangle the data and transform the shape of the data
into the model we mentioned earlier. The output should be a list of dictionaries
that look like this:
{
"id": "2406124091",
"type: "node",
"visible":"true",
"created": {
          "version":"2",
          "changeset":"17206049",
          "timestamp":"2013-08-03T16:43:42Z",
          "user":"linuxUser16",
          "uid":"1219059"
        },
"pos": [41.9757030, -87.6921867],
"address": {
          "housenumber": "5157",
          "postcode": "60625",
          "street": "North Lincoln Ave"
        },
"amenity": "restaurant",
"cuisine": "mexican",
"name": "La Cabana De Don Luis",
"phone": "1 (773)-271-5176"
}
You have to complete the function 'shape_element'.
We have provided a function that will parse the map file, and call the function with the element
as an argument. You should return a dictionary, containing the shaped data for that element.
We have also provided a way to save the data in a file, so that you could use
mongoimport later on to import the shaped data into MongoDB. You could also do some cleaning
before doing that, like in the previous exercise, but for this exercise you just have to
shape the structure.
In particular the following things should be done:
- you should process only 2 types of top level tags: "node" and "way"
- all attributes of "node" and "way" should be turned into regular key/value pairs, except:
    - attributes in the CREATED array should be added under a key "created"
    - attributes for latitude and longitude should be added to a "pos" array,
      for use in geospacial indexing. Make sure the values inside "pos" array are floats
      and not strings.
- if second level tag "k" value contains problematic characters, it should be ignored
- if second level tag "k" value starts with "addr:", it should be added to a dictionary "address"
- if second level tag "k" value does not start with "addr:", but contains ":", you can process it
  same as any other tag.
- if there is a second ":" that separates the type/direction of a street,
  the tag should be ignored, for example:
<tag k="addr:housenumber" v="5158"/>
<tag k="addr:street" v="North Lincoln Avenue"/>
<tag k="addr:street:name" v="Lincoln"/>
<tag k="addr:street:prefix" v="North"/>
<tag k="addr:street:type" v="Avenue"/>
<tag k="amenity" v="pharmacy"/>
  should be turned into:
{...
"address": {
    "housenumber": 5158,
    "street": "North Lincoln Avenue"
}
"amenity": "pharmacy",
...
}
- for "way" specifically:
  <nd ref="305896090"/>
  <nd ref="1719825889"/>
should be turned into
"node_ref": ["305896090", "1719825889"]
"""


lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')
addresschars = re.compile(r'addr:(\w+)')

CREATED = [ "version", "changeset", "timestamp", "user", "uid"]

def shape_element(element):
    #node = defaultdict(set)
    node = {}
    if element.tag == "node" or element.tag == "way" :
        #create the dictionary based on exaclty the value in element attribute.
        node = {'created':{}, 'type':element.tag}
        for k in element.attrib:
            try:
                v = element.attrib[k]
            except KeyError:
                continue
            if k == 'lat' or k == 'lon':
                continue
            if k in CREATED:
                node['created'][k] = v
            else:
                node[k] = v
        try:
            node['pos']=[float(element.attrib['lat']),float(element.attrib['lon'])]
        except KeyError:
            pass
        
        if 'address' not in node.keys():
            node['address'] = {}
        #Iterate the content of the tag
        for stag in element.iter('tag'):
            #Init the dictionry

            k = stag.attrib['k']
            v = stag.attrib['v']
            #Checking if indeed prefix with 'addr' and no ':' afterwards
            if k.startswith('addr:'):
                if len(k.split(':')) == 2:
                    content = addresschars.search(k)
                    if content:
                        node['address'][content.group(1)] = v
            else:
                node[k]=v
        if not node['address']:
            node.pop('address',None)
        #Special case when the tag == way,  scrap all the nd key
        if element.tag == "way":
            node['node_refs'] = []
            for nd in element.iter('nd'):
                node['node_refs'].append(nd.attrib['ref'])
#         if  'address' in node.keys():
#             pprint.pprint(node['address'])
        return node
    else:
        return None
    
def process_map(file_in, pretty = False):
    """
    Process the osm file to json file to be prepared for input file to monggo
    """
    file_out = "{0}.json".format(file_in)
    data = []
    with codecs.open(file_out, "w") as fo:
        for _, element in ET.iterparse(file_in):
            el = shape_element(element)
            if el:
                data.append(el)
                if pretty:
                    fo.write(json.dumps(el, indent=2)+"\n")
                else:
                    fo.write(json.dumps(el) + "\n")
    return data


In [47]:
Sample_data = process_map(SAMPLE_FILE)

In [48]:
pprint.pprint(Sample_data[0:6])

[{'created': {'changeset': '7781188',
              'timestamp': '2011-04-06T05:17:15Z',
              'uid': '207745',
              'user': 'NE2',
              'version': '54'},
  'exit_to': 'Joliet Road',
  'highway': 'motorway_junction',
  'id': '219850',
  'pos': [41.7585879, -87.9101245],
  'ref': '276C',
  'type': 'node'},
 {'created': {'changeset': '15349376',
              'timestamp': '2013-03-13T10:19:42Z',
              'uid': '439046',
              'user': 'Eliyak',
              'version': '12'},
  'id': '701638',
  'pos': [41.9850412, -87.8670872],
  'type': 'node'},
 {'created': {'changeset': '2090621',
              'timestamp': '2009-08-09T21:18:19Z',
              'uid': '1295',
              'user': 'robert',
              'version': '59'},
  'id': '739219',
  'pos': [41.5356981, -87.3112762],
  'type': 'node'},
 {'created': {'changeset': '8323836',
              'timestamp': '2011-06-02T21:10:41Z',
              'uid': '451048',
              'user': 'bbmiller',


In [49]:
client = MongoClient('localhost', 27017)

In [50]:
db = client.test_osm

In [51]:
db.osm_sample.insert(Sample_data)

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  if __name__ == '__main__':


[ObjectId('58028fc25536b531fb1f537e'),
 ObjectId('58028fc25536b531fb1f537f'),
 ObjectId('58028fc25536b531fb1f5380'),
 ObjectId('58028fc25536b531fb1f5381'),
 ObjectId('58028fc25536b531fb1f5382'),
 ObjectId('58028fc25536b531fb1f5383'),
 ObjectId('58028fc25536b531fb1f5384'),
 ObjectId('58028fc25536b531fb1f5385'),
 ObjectId('58028fc25536b531fb1f5386'),
 ObjectId('58028fc25536b531fb1f5387'),
 ObjectId('58028fc25536b531fb1f5388'),
 ObjectId('58028fc25536b531fb1f5389'),
 ObjectId('58028fc25536b531fb1f538a'),
 ObjectId('58028fc25536b531fb1f538b'),
 ObjectId('58028fc25536b531fb1f538c'),
 ObjectId('58028fc25536b531fb1f538d'),
 ObjectId('58028fc25536b531fb1f538e'),
 ObjectId('58028fc25536b531fb1f538f'),
 ObjectId('58028fc25536b531fb1f5390'),
 ObjectId('58028fc25536b531fb1f5391'),
 ObjectId('58028fc25536b531fb1f5392'),
 ObjectId('58028fc25536b531fb1f5393'),
 ObjectId('58028fc25536b531fb1f5394'),
 ObjectId('58028fc25536b531fb1f5395'),
 ObjectId('58028fc25536b531fb1f5396'),
 ObjectId('58028fc25536b5

In [55]:
def find():
    nodes = db.osm_sample.find({'id':'20217150'})
    for a in nodes:
        pprint.pprint(a)
        
find()

{u'_id': ObjectId('58028fc25536b531fb1f5381'),
 u'created': {u'changeset': u'8323836',
              u'timestamp': u'2011-06-02T21:10:41Z',
              u'uid': u'451048',
              u'user': u'bbmiller',
              u'version': u'4'},
 u'id': u'20217150',
 u'pos': [41.8887567, -87.6144004],
 u'source': u'PGS',
 u'type': u'node'}


In [68]:
#Show 5 data that have street
def street_5():
    result  = db.osm_sample.aggregate([
            {'$match': {'address.street':{'$exists':1}}},
            {'$limit' : 5}
        ])
    for a in result:
        pprint.pprint(a)

street_5()

{u'_id': ObjectId('58028fc25536b531fb1f5ff3'),
 u'address': {u'city': u'Skokie',
              u'housenumber': u'5001',
              u'postcode': u'60077',
              u'street': u'West Dempster Street'},
 u'amenity': u'cafe',
 u'created': {u'changeset': u'41319418',
              u'timestamp': u'2016-08-08T09:19:09Z',
              u'uid': u'1237170',
              u'user': u'Zol87',
              u'version': u'5'},
 u'cuisine': u'coffee_shop',
 u'id': u'310502167',
 u'name': u'Starbucks',
 u'pos': [42.04049, -87.751878],
 u'type': u'node',
 u'website': u'http://www.starbucks.com/store/14204/'}
{u'_id': ObjectId('58028fc25536b531fb1f6086'),
 u'address': {u'housenumber': u'640',
              u'postcode': u'60613',
              u'street': u'West Irving Park Road'},
 u'amenity': u'school',
 u'created': {u'changeset': u'10077676',
              u'timestamp': u'2011-12-10T00:02:35Z',
              u'uid': u'567034',
              u'user': u'Umbugbene',
              u'version': u'2'},

In [69]:
#Show the top 5 of contributed users
def users_5():
    result  = db.osm_sample.aggregate([
            {'$match': {'created.user':{'$exists':1}}},
            {'$group': {'_id':'$created.user',
                        'count':{'$sum':1}}},
            {'$sort': {'count':-1}},
            {'$limit' : 5}
        ])
    for a in result:
        pprint.pprint(a)

users_5()

{u'_id': u'chicago-buildings', u'count': 37588}
{u'_id': u'Umbugbene', u'count': 7300}
{u'_id': u'alexrudd (NHD)', u'count': 1531}
{u'_id': u'woodpeck_fixbot', u'count': 1528}
{u'_id': u'patester24', u'count': 752}


In [70]:
#Show the restaurant's name, the food they serve, and contact number

def restaurant_5():
    result  = db.osm_sample.aggregate([
           {'$match': {'amenity':'restaurant',
                        'name':{'$exists':1}}},
            {'$project':{'_id':'$name',
                         'cuisine':'$cuisine',
                         'contact':'$phone'}}
        ])
    for a in result:
        pprint.pprint(a)

restaurant_5()


{u'_id': u"Aurelio's Pizza"}
{u'_id': u'Sandwich Me In', u'cuisine': u'sandwich'}
{u'_id': u'Aguas Tortas', u'cuisine': u'mexican'}
{u'_id': u'Barba Yianni', u'cuisine': u'greek'}
{u'_id': u'Taylor Street Pizza', u'cuisine': u'pizza'}
{u'_id': u'Local Root'}
{u'_id': u'Creamery', u'cuisine': u'ice_cream'}
{u'_id': u"Elly's Pancake House", u'cuisine': u'pancake'}
{u'_id': u'Moy Lee Chinese Restaurant', u'cuisine': u'chinese'}
{u'_id': u"Brown's Chicken"}
